In [1]:
%matplotlib inline
import pandas as pd

#Importo Decisions.csv
decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Da "Decisions.csv" seleziono solo le tuple riguardanti il round desiderato (1-9)
decisions1 = decisions[decisions["Round"] == 1]

#Ho effettuato un controllo esterno per assicurarmi che tutte le tuple vengano prese in considerazione

#Estraggo "ID", "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions1 = decisions1.iloc[:, [0, 1, 3, 4]]

#Da "Decisions.csv" seleziono solo le tuple riguardanti il round successivo
decisions2 = decisions[decisions["Round"] == 2]

#Estraggo "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions2 = decisions2.iloc[:, [1, 3, 4]]

#Importo TasksQualities per le task assegnate a ciascun WA
queue = pd.read_csv("../Dataset 2.0/TasksQualities.csv", sep = ";")

queue = queue.sort_values(by = "ID")


print("end")

end


In [2]:
from sklearn.linear_model import LinearRegression
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


#Creo un DataFrame vuoto nel quale concatenerò il risultato di ciascuna iterazione che sto per eseguire
totale = pd.DataFrame()

#Creo un DataFrame vuoto nel quale concatenerò i coefficienti di regressione per ogni WA
coeffs = pd.DataFrame()


for i in range(20):
    
    #Faccio coincidere in entrambe le tabelle gli wa scelti (in due round consecutivi)
    tmp1 = decisions1[decisions1["Worker Agent ID"] == i+1]
    tmp2 = decisions2[decisions2["Worker Agent ID"] == i+1]
    tmp1 = pd.merge(tmp1, tmp2, left_on = "Session ID", right_on = "Session ID")
    tmp1 = tmp1.rename(columns = {"Worker Agent ID_x": "Worker Agent ID", "Worker Agent Reputation_x": "Worker Agent Starting Reputation", "Worker Agent Reputation_y": "Worker Agent Observed Reputation"})
    tmp1 = pd.merge(queue, tmp1, left_on = "ID", right_on = "ID")
    tmp1 = tmp1.drop("Worker Agent ID_y", 1)
    tmp1 = tmp1.drop("Session ID", 1)
    tmp1 = tmp1.sort_values(by = "ID")
    
    
    #Preparazione e training del modello per ogni wa riguardo al round scelto
    xtmp = tmp1.iloc[:, 0:93]
    ytmp = tmp1.iloc[:, 93]
    
    reg = LinearRegression().fit(xtmp, ytmp)
    
    
    #Salvataggio dei coefficienti in un Dataframe Unico
    var = pd.DataFrame(reg.coef_, columns = [f"{i + 1}"])

    totale = pd.concat([totale, tmp1])
    
    coeffs[f"{i+1}"] =  var
    coeffs.to_csv("../Coefficienti di Regressione/coeffs.csv", index = False)
    
print("end")

end


In [3]:
totale = totale.sort_values(by = "ID")

#Preparazione del dataset per il training
x = totale.iloc[:, 0:93]
y = totale.iloc[:, 93]
x

,ID,value1,difficulty1,er1,value2,difficulty2,er2,value3,difficulty3,er3,...,difficulty28,er28,value29,difficulty29,er29,value30,difficulty30,er30,Worker Agent ID,Worker Agent Starting Reputation
0,1,4,0.8,4,2,0.4,2,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,1,0.5
0,3,0,0.0,0,0,0.0,0,5,1,5,...,0.0,0,0,0.0,0,0,0.0,0,6,0.5
0,5,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,3,0.5
0,7,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,5,0.5
0,9,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,8,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4334,1290846,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,6,0.5
4334,1290848,4,0.8,4,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,7,0.5
4332,1290850,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,8,0.5
4332,1290852,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,9,0.5


In [4]:
#Training del modello utilizzando la Regressione Lineare

reg = LinearRegression().fit(x, y)
reg.score(x, y)

0.2394294578681999

In [5]:
coeffs

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,1.135297e-08,2.895223e-09,-5.525225e-09,7.610982e-09,-1.040070e-09,-4.327087e-09,4.564795e-09,-2.731863e-09,-1.225825e-09,-1.333201e-09,7.771331e-10,1.029156e-08,-6.599144e-09,-5.991374e-10,8.095032e-10,-3.547793e-09,2.438035e-09,8.025340e-10,-5.568801e-10,-1.865818e-09
1,-1.467271e-03,-6.051352e-03,2.910450e-04,1.088817e-03,2.351017e-03,2.802431e-03,3.383489e-03,4.996739e-03,5.676590e-03,2.933651e-03,-6.638590e-03,-6.619619e-03,-2.803775e-03,4.317027e-04,-2.020762e-03,9.169540e-04,3.384114e-03,4.435585e-03,5.338799e-03,1.232925e-03
2,-2.934543e-04,-1.210270e-03,5.820901e-05,2.177633e-04,4.702033e-04,5.604861e-04,6.766979e-04,9.993478e-04,1.135318e-03,5.867302e-04,-1.327718e-03,-1.323924e-03,-5.607550e-04,8.634053e-05,-4.041525e-04,1.833908e-04,6.768228e-04,8.871171e-04,1.067760e-03,2.465849e-04
3,-1.467271e-03,-6.051352e-03,2.910450e-04,1.088817e-03,2.351017e-03,2.802431e-03,3.383489e-03,4.996739e-03,5.676590e-03,2.933651e-03,-6.638590e-03,-6.619619e-03,-2.803775e-03,4.317027e-04,-2.020762e-03,9.169540e-04,3.384114e-03,4.435585e-03,5.338799e-03,1.232925e-03
4,-6.104951e-03,2.825299e-03,1.157873e-02,1.648174e-02,1.207603e-02,1.153152e-02,1.066671e-02,1.071478e-02,9.064550e-03,7.261920e-03,-2.357339e-03,3.108167e-03,5.176090e-03,1.089036e-02,1.415033e-02,1.227756e-02,9.092171e-03,1.061560e-02,9.175133e-03,3.269367e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,-5.604477e-03,8.582621e-04,1.400089e-02,1.681026e-02,1.694526e-02,1.474596e-02,1.113068e-02,9.048374e-03,9.588731e-03,9.501525e-03,-7.889762e-03,-1.462658e-03,1.068692e-02,1.543930e-02,1.389201e-02,1.374094e-02,1.272761e-02,1.071478e-02,6.892532e-03,4.133913e-03
89,-1.120895e-03,1.716524e-04,2.800178e-03,3.362052e-03,3.389051e-03,2.949192e-03,2.226137e-03,1.809675e-03,1.917746e-03,1.900305e-03,-1.577952e-03,-2.925317e-04,2.137383e-03,3.087859e-03,2.778401e-03,2.748189e-03,2.545522e-03,2.142956e-03,1.378506e-03,8.267827e-04
90,-5.604477e-03,8.582621e-04,1.400089e-02,1.681026e-02,1.694526e-02,1.474596e-02,1.113068e-02,9.048374e-03,9.588731e-03,9.501525e-03,-7.889762e-03,-1.462658e-03,1.068692e-02,1.543930e-02,1.389201e-02,1.374094e-02,1.272761e-02,1.071478e-02,6.892532e-03,4.133913e-03
91,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
#Training del modello utilizzando SVR

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(x, y)
regr.score(x, y)